In [15]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [27]:
IMG_NAME = 'code_jeje_yang/Submit/images/DZ6079a_02A05 DAPI 10x.tif'

In [ ]:
res = detectCircles(dilation,10,1)

In [ ]:
def detectCircles(img,threshold,region,radius = None):
    (M,N) = img.shape
    if radius == None:
        R_max = np.max((M,N))
        R_min = 3
    else:
        [R_max,R_min] = radius

    R = R_max - R_min
    #Initializing accumulator array.
    #Accumulator array is a 3 dimensional array with the dimensions representing
    #the radius, X coordinate and Y coordinate resectively.
    #Also appending a padding of 2 times R_max to overcome the problems of overflow
    A = np.zeros((R_max,M+2*R_max,N+2*R_max))
    B = np.zeros((R_max,M+2*R_max,N+2*R_max))

    #Precomputing all angles to increase the speed of the algorithm
    theta = np.arange(0,360)*np.pi/180
    edges = np.argwhere(img[:,:])                                               #Extracting all edge coordinates
    for val in range(R):
        r = R_min+val
        #Creating a Circle Blueprint
        bprint = np.zeros((2*(r+1),2*(r+1)))
        (m,n) = (r+1,r+1)                                                       #Finding out the center of the blueprint
        for angle in theta:
            x = int(np.round(r*np.cos(angle)))
            y = int(np.round(r*np.sin(angle)))
            bprint[m+x,n+y] = 1
        constant = np.argwhere(bprint).shape[0]
        for x,y in edges:                                                       #For each edge coordinates
            #Centering the blueprint circle over the edges
            #and updating the accumulator array
            X = [x-m+R_max,x+m+R_max]                                           #Computing the extreme X values
            Y= [y-n+R_max,y+n+R_max]                                            #Computing the extreme Y values
            A[r,X[0]:X[1],Y[0]:Y[1]] += bprint
        A[r][A[r]<threshold*constant/r] = 0

    for r,x,y in np.argwhere(A):
        temp = A[r-region:r+region,x-region:x+region,y-region:y+region]
        try:
            p,a,b = np.unravel_index(np.argmax(temp),temp.shape)
        except:
            continue
        B[r+(p-region),x+(a-region),y+(b-region)] = 1

    return B[:,R_max:-R_max,R_max:-R_max]

In [60]:
# edges1 = cv2.resize(edges1, (1000, 700))
# edges2 = cv2.resize(edges2, (1000, 700))
# dilation = cv2.resize(dilation, (1000, 700))

cv2.imshow('dilation', dilation)
# cv2.imshow('edges1', image)
# cv2.imshow('edges2', edges2)
cv2.waitKey(0)
cv2.destroyAllWindows()

# test opencv

In [107]:
def detect_circle(img_name):
    img = cv2.imread(img_name, 0)
    color_img = cv2.imread(img_name)

    ret, img = cv2.threshold(img, 120, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)      

    edges1 = cv2.Canny(img, 10, 100)
    edges2 = cv2.Canny(img, 10, 150)

    horizon_kernel = np.ones((1,3), np.uint8)
    vertical_kernel = np.ones((3,1), np.uint8)

    dilation = cv2.dilate(edges2,horizon_kernel,iterations = 1)
    dilation = cv2.dilate(dilation,vertical_kernel,iterations = 1)
    
#     dilation = cv2.bitwise_not(dilation)
    circles = cv2.HoughCircles(image=dilation, 
                               method=cv2.HOUGH_GRADIENT,
                               dp=1.5,
                               minDist=50,)
#                                param1=50,
#                                param2=50,
#                                minRadius=10,
#                                maxRadius=-1)
    
    for i in circles[0,:]:
        # draw the center of the circle
        cv2.circle(color_img,(i[0],i[1]), 1, (0,255,0), 1)
        
    cv2.imshow('detected circles', color_img)
    cv2.imshow('dilation', dilation)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [108]:
detect_circle(IMG_NAME)